# Coleta, limpeza e checagem dos dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import numpy as np

## Despesas por função

O código utilizado para coleta, via *Selenium*, está disponível abaixo como comentário. Neste arquivo apenas realizamos a leitura e tratamento dos dados previamente coletados. 


Código utilizado para coleta (para saber mais sobre as configurações da biblioteca utilizada, consulte [este link](https://pypi.org/project/selenium/): 


``` python 
import pandas as pd

from selenium.webdriver import Chrome, ChromeOptions, Firefox
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import time 
from selenium.webdriver.firefox.options import Options


for a in range(2010,2023):
    print(a)
    meses = ['dez', 'nov', 'out', 'set', 'ago', 'jul', 'jun', 'mai', 'abr','mar', 'fev', 'jan']
    colunas = ['descrição da despesa','Valor Anual Previsto na LOA','valor mensal empenhado','Valor Acumulado']
    df_ano_mes = []
    for i,m in enumerate(meses):
        print(m)
        url = f'https://transparencia.fortaleza.ce.gov.br/index.php/despesa/exibirDespesasPorFuncao/{a}#tabs-{m}'

        options = Options()
        options.headless = True
        webdriver = Firefox(executable_path='/home/alexandre/bin/geckodriver', options=options)

        webdriver.get(url)
        time.sleep(3)

        table = webdriver.find_elements(By.CLASS_NAME, 'table')[11-i]
        
        rows = table.find_elements(By.TAG_NAME, "tr") 
        cr = 0
        lista_dados = []
        for row in rows[1:]:
            col = row.find_elements(By.TAG_NAME, "td")
            lista_dados.append({
                                'ano':a,
                                'mes':m,
                                colunas[0]:col[0].text,
                                colunas[1]:col[1].text,
                                colunas[2]:col[2].text,
                                colunas[3]:col[3].text,
                                })
        df = pd.DataFrame(lista_dados)

        df_ano_mes.append(df)

        df = pd.concat(df_ano_mes)
        df.to_csv(f'df_{a}.csv')
        webdriver.close()
        
```




In [ ]:
dict_meses = {'dez':12,
 'nov':11, 
 'out':10,
 'set':9, 
 'ago':8, 
 'jul':7, 
 'jun':6,
 'mai':5,
 'abr':4,
 'mar':3,
 'fev':2,
 'jan':1
 }

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
!pip install calculadora_do_cidadao

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from calculadora_do_cidadao import Ipca
from datetime import date

In [ ]:
ipca = Ipca()

In [ ]:
path_scrap = '/content/drive/Shareddrives/O POVO+/Jornalismo de Dados/Pautas/2022/02 - Fevereiro/Execução Orçamentária PMF - Cultura/web_scrap_selenium'

In [ ]:
lista_df = []

for f in os.listdir(path_scrap):
  lista_df.append(pd.read_csv(path_scrap+'/'+f))

wsDf = pd.concat(lista_df)[['ano', 'mes', 'descrição da despesa',
       'Valor Anual Previsto na LOA', 'valor mensal empenhado',
       'Valor Acumulado']]

In [ ]:
wsDf['ano/mes'] = wsDf.apply(lambda x: f'{x.mes}/{x.ano}', axis=1)

In [ ]:
wsDf['Valor Anual Previsto na LOA'] = wsDf['Valor Anual Previsto na LOA'].str.replace('.', '').str.replace(',', '.').astype(float)
wsDf['valor mensal empenhado'] = wsDf['valor mensal empenhado'].str.replace('.', '').str.replace(',', '.').astype(float)
wsDf['Valor Acumulado'] = wsDf['Valor Acumulado'].str.replace('.', '').str.replace(',', '.').astype(float)

<ipython-input-109-00861aabf46c>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  wsDf['Valor Anual Previsto na LOA'] = wsDf['Valor Anual Previsto na LOA'].str.replace('.', '').str.replace(',', '.').astype(float)
<ipython-input-109-00861aabf46c>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  wsDf['valor mensal empenhado'] = wsDf['valor mensal empenhado'].str.replace('.', '').str.replace(',', '.').astype(float)
<ipython-input-109-00861aabf46c>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  wsDf['Valor Acumulado'] = ws

In [ ]:
wsDf['dtg'] = wsDf.apply(lambda x: pd.to_datetime(f'{x.ano}-{dict_meses[x.mes]}'), axis=1)

In [ ]:
wsDf['valor mensal empenhado corrigido'] = wsDf.apply(lambda x: x['valor mensal empenhado'] * float(ipca.adjust(x['dtg'])), axis=1)
wsDf['valor Acumulado corrigido'] = wsDf.apply(lambda x: x['Valor Acumulado'] * float(ipca.adjust(x['dtg'])), axis=1)

In [ ]:
wsDf.loc[wsDf['descrição da despesa'] != 'TOTAIS'][['ano','mes','valor mensal empenhado','valor mensal empenhado corrigido','dtg']]

,ano,mes,valor mensal empenhado,valor mensal empenhado corrigido,dtg
0,2010,dez,4716215.96,9604529.55,2010-12-01
1,2010,dez,3171516.62,6458763.84,2010-12-01
2,2010,dez,42307955.78,86159755.00,2010-12-01
3,2010,dez,7338359.70,14944500.68,2010-12-01
4,2010,dez,7098174.92,14455366.63,2010-12-01
...,...,...,...,...,...
294,2022,jan,0.00,0.00,2022-01-01
295,2022,jan,8185249.74,8657906.74,2022-01-01
296,2022,jan,2304339.12,2437403.11,2022-01-01
297,2022,jan,30325251.27,32076382.01,2022-01-01


## Despesas por área de atuação

A coleta dos dados foi feita com o webscraper, extensão para browsers. Para saber mais sobre a ferramenta, acesse [este link](https://www.webscraper.io/).

In [ ]:
path = '/content/drive/Shareddrives/O POVO+/Jornalismo de Dados/Pautas/2022/02 - Fevereiro/Execução Orçamentária PMF - Cultura/'

In [ ]:
df = pd.read_csv(path + 'despesas_por_area_de_atuacao.csv')

In [ ]:
funcoes = ['ADMINISTRACAO', 'ASSISTENCIA SOCIAL', 'CIENCIA E TECNOLOGIA', 'COMERCIO E SERVICOS', 'COMUNICACOES',  'CULTURA', 'DESPORTO E LAZER', 'DIREITOS DA CIDADANIA', 'EDUCACAO', 'ENCARGOS ESPECIAIS', 'ENERGIA', 'ESSENCIAL A JUSTICA', 'GESTAO AMBIENTAL', 'HABITACAO', 'JUDICIARIA', 'LEGISLATIVA', 'PREVIDENCIA SOCIAL', 'RESERVA DE CONTINGENCIA', 'SANEAMENTO', 'SAUDE', 'SEGURANCA PUBLICA', 'TRABALHO', 'TRANSPORTE', 'URBANISMO']

In [ ]:
df['tipo'] = np.where(df['função / subfunção'].isin(funcoes), 'função', 'subfunção')

In [ ]:
df['ano'] = df['web-scraper-start-url'].apply(lambda x: x.split("exercicio=")[1].split("&")[0]) 

In [ ]:
df['Dotação Orçamentária'] = df['Dotação Orçamentária'].str.replace('.', '').str.replace(',', '.').astype(float)
df['Valor Empenhado'] = df['Valor Empenhado'].str.replace('.', '').str.replace(',', '.').astype(float)
df['Valor Liquidado'] = df['Valor Liquidado'].str.replace('.', '').str.replace(',', '.').astype(float)
df['Valor Pago'] = df['Valor Pago'].str.replace('.', '').str.replace(',', '.').astype(float)

<ipython-input-118-758bbebceb65>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Dotação Orçamentária'] = df['Dotação Orçamentária'].str.replace('.', '').str.replace(',', '.').astype(float)
<ipython-input-118-758bbebceb65>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Valor Empenhado'] = df['Valor Empenhado'].str.replace('.', '').str.replace(',', '.').astype(float)
<ipython-input-118-758bbebceb65>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Valor Liquidado'] = df['Valor Liquidado'].str.replace('.', '

In [ ]:
df['dtg'] = df.ano.apply(lambda x: pd.to_datetime(f'{x}-12-31')) 

In [ ]:
df['valor_pago_corrigido'] = df.apply(lambda x: x['Valor Pago'] * float(ipca.adjust(x['dtg'])), axis=1)
df['valor_empenhado_corrigido'] = df.apply(lambda x: x['Valor Empenhado'] * float(ipca.adjust(x['dtg'])), axis=1)
df['valor_liquidado_corrigido'] = df.apply(lambda x: x['Valor Liquidado'] * float(ipca.adjust(x['dtg'])), axis=1)

In [ ]:
def getSubfunc(idx_ano):
  cont = idx_ano+1
  while df.iloc[cont].tipo == 'subfunção':
    cont+=1
  return df.iloc[idx_ano+1:cont].copy()

In [ ]:
lista_subfuncoes = []
for y in list(df[df['função / subfunção'] == 'CULTURA'].index):
  lista_subfuncoes.append(getSubfunc(y))

In [ ]:
subfuncoes_cultura = pd.concat(lista_subfuncoes)

In [ ]:
subfuncoes_cultura

,web-scraper-order,web-scraper-start-url,função / subfunção,Dotação Orçamentária,Valor Empenhado,%(B/A)%,Valor Liquidado,Valor Pago,tipo,ano,dtg,valor_pago_corrigido,valor_empenhado_corrigido,valor_liquidado_corrigido
71,1676496995-491,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,41801420.00,26512144.03,"63,42",20563025.45,19885172.83,subfunção,2022,2022-12-31,19990556.02,26652647.43,20672000.98
72,1676496995-492,https://transparencia.fortaleza.ce.gov.br/inde...,TECNOLOGIA DA INFORMACAO,11000.00,0.00,"0,00",0.00,0.00,subfunção,2022,2022-12-31,0.00,0.00,0.00
73,1676496995-493,https://transparencia.fortaleza.ce.gov.br/inde...,ASSISTENCIA A CRIANCA E AO ADOLESCENTE,7384954.00,7086685.44,"95,96",7086685.44,5904003.35,subfunção,2022,2022-12-31,5935292.13,7124241.94,7124241.94
74,1676496995-494,https://transparencia.fortaleza.ce.gov.br/inde...,"PATRIMONIO HISTORICO, ARTISTICO E ARQUEOLOGICO",3432954.00,611288.38,"17,81",611288.38,611288.38,subfunção,2022,2022-12-31,614527.96,614527.96,614527.96
75,1676496995-495,https://transparencia.fortaleza.ce.gov.br/inde...,DIFUSAO CULTURAL,45825490.00,7887471.91,"17,21",7887471.91,7726750.73,subfunção,2022,2022-12-31,7767699.31,7929272.25,7929272.25
207,1676496998-627,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,23992607.00,18593628.52,"77,50",16668781.43,16373598.47,subfunção,2021,2021-12-31,17412619.57,19773526.29,17726533.99
208,1676496998-628,https://transparencia.fortaleza.ce.gov.br/inde...,TECNOLOGIA DA INFORMACAO,1134000.00,100382.60,"8,85",100382.60,90065.60,subfunção,2021,2021-12-31,95780.90,106752.59,106752.59
209,1676496998-629,https://transparencia.fortaleza.ce.gov.br/inde...,ASSISTENCIA A CRIANCA E AO ADOLESCENTE,4195000.00,4169214.13,"99,39",4169214.13,3169214.13,subfunção,2021,2021-12-31,3370323.27,4433780.37,4433780.37
210,1676496998-630,https://transparencia.fortaleza.ce.gov.br/inde...,"PATRIMONIO HISTORICO, ARTISTICO E ARQUEOLOGICO",1059852.00,740346.72,"69,85",343190.23,343190.23,subfunção,2021,2021-12-31,364968.09,787326.98,364968.09
211,1676496998-631,https://transparencia.fortaleza.ce.gov.br/inde...,DIFUSAO CULTURAL,46327302.00,36993107.83,"79,85",34803470.15,34647161.49,subfunção,2021,2021-12-31,36845769.94,39340583.23,37011997.49


# Análise e visualizações

## Valor empenhado para a Cultura por ano em Fortaleza-CE, entre 2010 e 2022 e Percentual do valor do orçamento empenhado para o setor cultural em relação às demais funções de governo entre 2010 e 2022


In [ ]:
resumo_cultura_total = wsDf.loc[(wsDf['descrição da despesa'] == 'TOTAIS') | (wsDf['descrição da despesa'] == 'CULTURA')].pivot(index=['ano','mes'], columns ='descrição da despesa', values = 'valor mensal empenhado corrigido').reset_index()

In [ ]:
resumo_cultura_total['pct'] = round(resumo_cultura_total['CULTURA']/resumo_cultura_total['TOTAIS']*100,2)

In [ ]:
resumo_cultura_total.columns = ['Ano','Mẽs', 'Cultura', 'Todas as funções', 'Pct'] 

In [ ]:
resumo_cultura_total

,Ano,Mẽs,Cultura,Todas as funções,Pct
0,2010,abr,6965212.81,547477258.00,1.27
1,2010,ago,2862063.61,579649214.11,0.49
2,2010,dez,18691203.23,799663038.78,2.34
3,2010,fev,6995894.50,509875875.77,1.37
4,2010,jan,3968270.95,504298148.90,0.79
...,...,...,...,...,...
151,2022,mai,6156187.13,928150466.34,0.66
152,2022,mar,1427653.13,1116884716.88,0.13
153,2022,nov,17103369.85,1090272213.07,1.57
154,2022,out,3193034.68,902331734.78,0.35


In [ ]:
resumo_cultura_total_ano =  wsDf.loc[((wsDf['descrição da despesa'] == 'TOTAIS') | (wsDf['descrição da despesa'] == 'CULTURA')) & (wsDf['mes'] == 'dez')].pivot(index='ano', columns ='descrição da despesa', values = 'valor Acumulado corrigido').reset_index()

In [ ]:
resumo_cultura_total_ano['pct'] = round(resumo_cultura_total_ano['CULTURA']/resumo_cultura_total_ano['TOTAIS']*100,2)

In [ ]:
resumo_cultura_total_ano.columns = ['Ano', 'Cultura', 'Todas as funções', 'Pct']

In [ ]:
resumo_cultura_total_ano

,Ano,Cultura,Todas as funções,Pct
0,2010,63574897.77,6854758774.67,0.93
1,2011,79853713.07,7434734662.44,1.07
2,2012,90748840.93,7956966069.52,1.14
3,2013,62641182.40,8208514855.90,0.76
4,2014,71851622.03,8680770506.07,0.83
5,2015,46722825.51,8470634666.43,0.55
6,2016,63281835.17,8239379085.96,0.77
7,2017,57383665.53,8638652889.27,0.66
8,2018,76637695.50,8774388088.92,0.87
9,2019,88458631.69,10040407237.48,0.88


In [ ]:
resumo_cultura_total_ano_sem_correcao =  wsDf.loc[((wsDf['descrição da despesa'] == 'TOTAIS') | (wsDf['descrição da despesa'] == 'CULTURA')) & (wsDf['mes'] == 'dez')].pivot(index='ano', columns ='descrição da despesa', values = 'Valor Acumulado').reset_index()

In [ ]:
resumo_cultura_total_ano_sem_correcao['pct'] = round(resumo_cultura_total_ano_sem_correcao['CULTURA']/resumo_cultura_total_ano_sem_correcao['TOTAIS']*100,2)
resumo_cultura_total_ano_sem_correcao.columns = ['Ano', 'Cultura _ Sem Correção', 'Todas as funções _ Sem correção', 'Pct  _ Sem correção']

In [ ]:
resumo_cultura_total_ano_sem_correcao

,Ano,Cultura _ Sem Correção,Todas as funções _ Sem correção,Pct _ Sem correção
0,2010,31217869.22,3365966292.85,0.93
1,2011,41761489.58,3888179800.35,1.07
2,2012,50230328.42,4404254807.14,1.14
3,2013,36721858.05,4812040669.91,0.76
4,2014,44820159.52,5414958046.03,0.83
5,2015,32255792.19,5847827663.35,0.55
6,2016,46434616.84,6045848856.99,0.77
7,2017,43347750.02,6525657824.35,0.66
8,2018,60060690.19,6876456829.67,0.87
9,2019,72309943.34,8207466751.46,0.88


In [ ]:
pd.merge(resumo_cultura_total_ano_sem_correcao, resumo_cultura_total_ano, on='Ano')

,Ano,Cultura _ Sem Correção,Todas as funções _ Sem correção,Pct _ Sem correção,Cultura,Todas as funções,Pct
0,2010,31217869.22,3365966292.85,0.93,63574897.77,6854758774.67,0.93
1,2011,41761489.58,3888179800.35,1.07,79853713.07,7434734662.44,1.07
2,2012,50230328.42,4404254807.14,1.14,90748840.93,7956966069.52,1.14
3,2013,36721858.05,4812040669.91,0.76,62641182.40,8208514855.90,0.76
4,2014,44820159.52,5414958046.03,0.83,71851622.03,8680770506.07,0.83
5,2015,32255792.19,5847827663.35,0.55,46722825.51,8470634666.43,0.55
6,2016,46434616.84,6045848856.99,0.77,63281835.17,8239379085.96,0.77
7,2017,43347750.02,6525657824.35,0.66,57383665.53,8638652889.27,0.66
8,2018,60060690.19,6876456829.67,0.87,76637695.50,8774388088.92,0.87
9,2019,72309943.34,8207466751.46,0.88,88458631.69,10040407237.48,0.88


## Valor empenhado em Cultura em cada ano (2010-2022) em relação ao que estava previsto na Lei Orçamentária Anual (LOA)

In [ ]:
cultura_por_ano = wsDf.loc[(wsDf['descrição da despesa'] == 'CULTURA') &(wsDf['mes']== 'dez')].sort_values(by='ano')
cultura_por_ano['pct']  = round(cultura_por_ano['Valor Acumulado']/cultura_por_ano['Valor Anual Previsto na LOA']*100,2)
cultura_por_ano=cultura_por_ano[['ano','Valor Anual Previsto na LOA','Valor Acumulado','pct']]
cultura_por_ano.columns = ['Ano','Valor previsto na LOA','Valor empenhado no ano','Pct']
cultura_por_ano.to_csv('valor_anual_previsto_LOA_e_valor_acumulado.csv', index=False)
cultura_por_ano

,Ano,Valor previsto na LOA,Valor empenhado no ano,Pct
9,2010,63770546.00,31217869.22,48.95
9,2011,78348304.00,41761489.58,53.30
9,2012,99861612.00,50230328.42,50.30
9,2013,90266544.00,36721858.05,40.68
11,2014,57645418.00,44820159.52,77.75
10,2015,57985508.00,32255792.19,55.63
10,2016,88398641.00,46434616.84,52.53
10,2017,96365773.00,43347750.02,44.98
10,2018,95069137.00,60060690.19,63.18
10,2019,104948694.00,72309943.34,68.90


## Valores Empenhados, Liquidados e Pagos para a Cultura, entre 2014 e 2022
(Valores totais e distribuídos por subfunção)


In [ ]:
valores_cultura_soma = subfuncoes_cultura.groupby('ano').sum().round(2)
valores_cultura_soma

,Dotação Orçamentária,Valor Empenhado,Valor Liquidado,Valor Pago,valor_pago_corrigido,valor_empenhado_corrigido,valor_liquidado_corrigido
ano,,,,,,,
2014,73248537.00,44820159.52,42945440.26,37758174.45,60530486.91,71851622.03,68846241.84
2015,55866508.00,32255792.19,32054672.20,29636524.95,42928791.70,46722825.51,46431501.27
2016,90139716.00,46434616.84,44820208.66,43775480.12,59657921.31,63281835.17,61081694.00
2017,79607923.00,43347750.02,41662131.35,41183168.28,54518196.51,57383665.53,55152246.88
2018,84024373.00,60060690.19,58453363.52,58239392.83,74313712.34,76637695.50,74586739.85
2019,90629635.00,72309943.34,68492245.07,68135466.38,83351885.60,88458631.69,83788342.24
2020,172910763.00,136001177.20,131205471.39,131051846.32,153389808.74,159182759.69,153569619.41
2021,100042367.00,83328486.20,75203332.04,73335155.17,77988791.56,88616270.41,79975517.52
2022,98455818.00,42097589.76,36148471.18,34127215.29,34308075.42,42320689.58,36340043.13


In [ ]:
valores_cultura = df[df['função / subfunção'] == 'CULTURA'][['ano','Dotação Orçamentária', 'Valor Empenhado','Valor Liquidado',
       'Valor Pago','%(B/A)%','valor_pago_corrigido','valor_empenhado_corrigido','valor_liquidado_corrigido']].sort_values(by='ano')
valores_cultura

,ano,Dotação Orçamentária,Valor Empenhado,Valor Liquidado,Valor Pago,%(B/A)%,valor_pago_corrigido,valor_empenhado_corrigido,valor_liquidado_corrigido
1144,2014,73248537.00,44820159.52,42945440.26,37758174.45,"61,19",60530486.91,71851622.03,68846241.84
1012,2015,55866508.00,32255792.19,32054672.20,29636524.95,"57,74",42928791.70,46722825.51,46431501.27
889,2016,90139716.00,46434616.84,44820208.66,43775480.12,"51,51",59657921.31,63281835.17,61081694.00
766,2017,79607923.00,43347750.02,41662131.35,41183168.28,"54,45",54518196.51,57383665.53,55152246.88
629,2018,84024373.00,60060690.19,58453363.52,58239392.83,"71,48",74313712.34,76637695.50,74586739.85
486,2019,90629635.00,72309943.34,68492245.07,68135466.38,"79,79",83351885.60,88458631.69,83788342.24
346,2020,172910763.00,136001177.20,131205471.39,131051846.32,"78,65",153389808.74,159182759.69,153569619.41
206,2021,100042367.00,83328486.20,75203332.04,73335155.17,"83,29",77988791.56,88616270.41,79975517.52
70,2022,98455818.00,42097589.76,36148471.18,34127215.29,"42,76",34308075.42,42320689.58,36340043.13


In [ ]:
# Teste de valores totais para cultura e divididos por subfunção, a cada ano:

valores_cultura_soma.reset_index()['Valor Pago'] == valores_cultura.reset_index()['Valor Pago']

0    True
1    True
2    True
3    True
4    True
5    True
6    True
7    True
8    True
Name: Valor Pago, dtype: bool

In [ ]:
valores_cultura_soma.reset_index()['Valor Liquidado'] == valores_cultura.reset_index()['Valor Liquidado']

0    True
1    True
2    True
3    True
4    True
5    True
6    True
7    True
8    True
Name: Valor Liquidado, dtype: bool

In [ ]:
valores_cultura_soma.reset_index()['Valor Empenhado'] == valores_cultura.reset_index()['Valor Empenhado']

0    True
1    True
2    True
3    True
4    True
5    True
6    True
7    True
8    True
Name: Valor Empenhado, dtype: bool

In [ ]:
valores_cultura_soma.reset_index()['Valor Pago'] == valores_cultura.reset_index()['Valor Pago']

0    True
1    True
2    True
3    True
4    True
5    True
6    True
7    True
8    True
Name: Valor Pago, dtype: bool

In [ ]:
colunas = ['ano','Dotação Orçamentária', 'Valor Empenhado', 'Valor Liquidado', 'Valor Pago','função / subfunção','valor_pago_corrigido','valor_empenhado_corrigido','valor_liquidado_corrigido']

todas = df[df['função / subfunção'] == 'CULTURA'][colunas].copy().sort_values(by='ano')
todas['função / subfunção'] = 'Todas'

pd.concat([todas,subfuncoes_cultura[colunas]]).to_csv('gastos_cultura_por_subfuncao.csv', index=False)


pd.concat([todas,subfuncoes_cultura[colunas]])

,ano,Dotação Orçamentária,Valor Empenhado,Valor Liquidado,Valor Pago,função / subfunção,valor_pago_corrigido,valor_empenhado_corrigido,valor_liquidado_corrigido
1144,2014,73248537.00,44820159.52,42945440.26,37758174.45,Todas,60530486.91,71851622.03,68846241.84
1012,2015,55866508.00,32255792.19,32054672.20,29636524.95,Todas,42928791.70,46722825.51,46431501.27
889,2016,90139716.00,46434616.84,44820208.66,43775480.12,Todas,59657921.31,63281835.17,61081694.00
766,2017,79607923.00,43347750.02,41662131.35,41183168.28,Todas,54518196.51,57383665.53,55152246.88
629,2018,84024373.00,60060690.19,58453363.52,58239392.83,Todas,74313712.34,76637695.50,74586739.85
...,...,...,...,...,...,...,...,...,...
1146,2014,21000.00,0.00,0.00,0.00,"PATRIMONIO HISTORICO, ARTISTICO E ARQUEOLOGICO",0.00,0.00,0.00
1147,2014,40519754.00,26687171.91,24812452.65,21695922.53,DIFUSAO CULTURAL,34780938.80,42782457.93,39777077.74
1148,2014,5140.00,0.00,0.00,0.00,"DIREITOS INDIVIDUAIS, COLETIVOS E DIFUSOS",0.00,0.00,0.00
1149,2014,2249000.00,173802.97,173802.97,173802.97,DESPORTO COMUNITARIO,278625.19,278625.19,278625.19


In [ ]:
resumo_por_subfuncao =  subfuncoes_cultura[colunas].copy()

resumo_por_subfuncao.loc[(resumo_por_subfuncao['função / subfunção'] != 'ADMINISTRACAO GERAL') & (resumo_por_subfuncao['função / subfunção'] != 'DIFUSAO CULTURAL'),'função / subfunção'] = 'Outras'

resumo_por_subfuncao = resumo_por_subfuncao.groupby(['ano','função / subfunção']).sum().reset_index().pivot(index='ano', columns='função / subfunção', values='valor_pago_corrigido').fillna(0).reset_index()

In [ ]:
resumo_por_subfuncao

função / subfunção,ano,ADMINISTRACAO GERAL,DIFUSAO CULTURAL,Outras
0,2014,25470922.92,34780938.80,278625.19
1,2015,30018991.68,12314443.42,595356.59
2,2016,36756515.19,22043917.97,857488.15
3,2017,36245948.28,16410129.76,1862118.48
4,2018,20019732.93,12631837.51,41662141.91
5,2019,46326785.61,28364211.14,8660888.85
6,2020,38618931.34,67482937.90,47287939.50
7,2021,17412619.57,36845769.94,23730402.05
8,2022,19990556.02,7767699.31,6549820.08


In [ ]:
resumo_por_subfuncao.to_csv('resumo_por_subfuncao.csv', index=False)

In [ ]:
resumo_por_subfuncao_v2 =  subfuncoes_cultura[colunas].copy()

resumo_por_subfuncao_v2 = resumo_por_subfuncao_v2.groupby(['ano','função / subfunção']).sum().reset_index().pivot(index='ano', columns='função / subfunção', values='valor_pago_corrigido').fillna(0).reset_index()

In [ ]:
resumo_por_subfuncao_v2.to_csv('resumo_por_subfuncao_v2.csv', index=False)

In [ ]:
resumo_por_subfuncao_v2

função / subfunção,ano,ADMINISTRACAO GERAL,ASSISTENCIA A CRIANCA E AO ADOLESCENTE,DESPORTO COMUNITARIO,DIFUSAO CULTURAL,"DIREITOS INDIVIDUAIS, COLETIVOS E DIFUSOS",FORMACAO DE RECURSOS HUMANOS,INFRA-ESTRUTURA URBANA,LAZER,"PATRIMONIO HISTORICO, ARTISTICO E ARQUEOLOGICO",TECNOLOGIA DA INFORMACAO,TECNOLOGIA DA INFORMATIZACAO
0,2014,25470922.92,0.00,278625.19,34780938.80,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2015,30018991.68,0.00,284312.33,12314443.42,0.00,0.00,0.00,0.00,195127.28,0.00,115916.98
2,2016,36756515.19,0.00,0.00,22043917.97,0.00,0.00,152912.32,0.00,704575.83,0.00,0.00
3,2017,36245948.28,0.00,0.00,16410129.76,0.00,0.00,1762061.33,0.00,70161.29,29895.86,0.00
4,2018,20019732.93,2716234.95,0.00,12631837.51,29430684.22,0.00,6854352.84,0.00,2660869.89,0.00,0.00
5,2019,46326785.61,4924075.88,0.00,28364211.14,0.00,0.00,3658824.96,0.00,75356.88,2631.13,0.00
6,2020,38618931.34,4418273.11,0.00,67482937.90,0.00,0.00,41313901.90,0.00,1555764.49,0.00,0.00
7,2021,17412619.57,3370323.27,0.00,36845769.94,0.00,0.00,19899329.79,0.00,364968.09,95780.90,0.00
8,2022,19990556.02,5935292.13,0.00,7767699.31,0.00,0.00,0.00,0.00,614527.96,0.00,0.00
